### NPSC psudeo-extension stands for NuPlan Scenario Collection. 
#### It is a pkl file with the naming convention `filename.npsc.pkl` that contains:
* The data structure of a dictionary of scenario modifications.
* The `DATASET_PARAMS` list of filters in order to extract the correct scenarios to modify.

In [1]:
import logging
import nest_asyncio

nest_asyncio.apply()

import torch

torch.cuda.is_available()

import os

from nuplan.planning.script.run_simulation import main as main_simulation

### If your paths arent set correctly in the environment, set them here manually.

In [ ]:
%env NUPLAN_DATA_ROOT=../../../../datasets/nuplan/dataset
%env NUPLAN_MAPS_ROOT=../../data/nuplan/maps
%env NUPLAN_EXP_ROOT=../../data/nuplan/exp
%env NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/
%pwd

In [3]:
import hydra
from nuplan.planning.script.utils import (
    set_default_path,
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# If set, use the env. variable to overwrite the default dataset and experiment paths
set_default_path()

# If set, use the env. variable to overwrite the Hydra config
CONFIG_PATH = os.getenv("NUPLAN_HYDRA_CONFIG_PATH", "config/simulation")

if os.environ.get("NUPLAN_HYDRA_CONFIG_PATH") is not None:
    CONFIG_PATH = os.path.join("../../../../", CONFIG_PATH)

if os.path.basename(CONFIG_PATH) != "simulation":
    CONFIG_PATH = os.path.join(CONFIG_PATH, "simulation")
CONFIG_NAME = "default_simulation"

In [4]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(
    os.getenv("NUPLAN_TUTORIAL_PATH", ""), "../nuplan/planning/script"
)
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

### Choose the experiment name you want to re-run

In [ ]:
import pickle
#pdm_closed_pdm_closed_cross_pure_scen_20
#pdm_closed_pdm_closed_seq_pure_scen_100
#pdm_closed_pdm_closed_merge_pure_scen_20
#pdm_closed_pdm_closed_diverge_pure_scen_1400
EXPERIMENT_NAME = "choose_saved_experiment_name"
modification_file_path = "npsc_files/" + EXPERIMENT_NAME + ".npsc.pkl"
with open(modification_file_path, "rb") as output_file:
    mod_list_dict = pickle.load(output_file)
    print(list(mod_list_dict.keys()))
    DATASET_PARAMS = pickle.load(output_file)

print(DATASET_PARAMS)

### Path to appropriate checkpoints listed. Change if wrong

In [6]:
urban_ckpt = "/home/ehdykhne/occ-gen/experiments/pretrained_checkpoints/urbandriver_checkpoint.ckpt"
gc_pgp_ckpt = "/home/ehdykhne/occ-gen/experiments/pretrained_checkpoints/gc_pgp_checkpoint.ckpt"
hybrid_ckpt = "/home/ehdykhne/occ-gen/experiments/pretrained_checkpoints/pdm_offset_checkpoint.ckpt"

### Prep first regime. Select appropriate planners and observations from comments below.

In [ ]:
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
print(simulation_hydra_paths.config_name)
cfg_occ = hydra.compose(
    config_name=simulation_hydra_paths.config_name,
    overrides=[
        # planner
        ################################################
        "planner=pdm_closed_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
        # planner occlusion
        ################################################
        "+occlusion_cfg.occlusion=true",
        "+occlusion_cfg.manager_type=wedge",  # options: [range, shadow, wedge]
        "+occlusion_cfg.uncloak_reaction_time=0.5",  # 0.1
        "+occlusion_cfg.notice_threshold=0.33",  # 0.1
        ################################################
        # obs
        ################################################
        f"observation.planner_type=pdm_closed",
        ################################################
        # obs occlusion
        ################################################
        f"observation.occlusion_cfg.occlusion=True",
        f"observation.occlusion_cfg.manager_type=wedge",
        f"+observation.occlusion_cfg.uncloak_reaction_time=0.5",
        f"+observation.occlusion_cfg.notice_threshold=0.33",
        ################################################
        # scenario modification
        "+modify_scenario_simulations=true",
        "+second_testing_round=true", 
        # unlike in the bulk running experiments ipynb file, we are testing all the scenarios.
        # #not just the ones that are OCCs, just in case different settings result in different scenarios having OCCs. 
        # If you want to restrict this, you can set the `scenarios_to_check` variable in the config like in the bulk running experiments file.        f"+modification_file_path={modification_file_path}",
        # "+remove_other_agents=true",
        ################################################
        # misc
        ################################################
        "+simulation=closed_loop_multiagent",  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents, closed_loop_multiagent]
        "worker=ray_distributed",  # [sequential, ray_distributed]
        "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
        *DATASET_PARAMS,
        ################################################
    ],
)

output_folder_reg_1 = cfg_occ.output_dir
print('output_folder_reg_1 = "' + output_folder_reg_1 + '"')

## Below are config options to run other types of planners and observations (observations being non-ego vehicles):

# planner: pdm_closed
##########################################
# "planner=pdm_closed_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
##########################################

# obs: pdm_closed
##########################################
# f"observation.planner_type=pdm_closed",
##########################################

# planner: urban driver
##########################################
# "planner=ml_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
# "model=urban_driver_open_loop_model",
# "planner.ml_planner.model_config=${model}",
# f"planner.ml_planner.checkpoint_path={urban_ckpt}",
##########################################

# obs: urban driver
##########################################
# "+observation.model=urban_driver_open_loop_model",
# f"observation.planner_type=ml",
# "+observation.model_config=${model}",
# f"observation.checkpoint_path={urban_ckpt}",
##########################################

# planner: gc_pgp
################################################
# "planner=ml_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
# "model=gc_pgp_model",
# "model.aggregator.pre_train=false",
# "planner.ml_planner.model_config=${model}",
# f"planner.ml_planner.checkpoint_path={gc_pgp_ckpt}",
###############################################

# obs: gc_pgp
###############################################
# "+observation.model_config=${model}",
# "+observation.model=gc_pgp_model",
# "+observation.model.aggregator.pre_train=false",
# f"observation.planner_type=ml",
# f"observation.checkpoint_path={gc_pgp_ckpt}",
################################################

### Now, we run the first regime

In [ ]:
# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
main_simulation(cfg_occ)

### Prep Second Regime

In [ ]:
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
print(simulation_hydra_paths.config_name)
cfg_occ = hydra.compose(
    config_name=simulation_hydra_paths.config_name,
    overrides=[
        # planner
        ################################################
        "planner=pdm_closed_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
        # planner occlusion
        ################################################
        "+occlusion_cfg.occlusion=true",
        "+occlusion_cfg.manager_type=wedge",  # options: [range, shadow, wedge]
        "+occlusion_cfg.uncloak_reaction_time=0.5",  # 0.1
        "+occlusion_cfg.notice_threshold=0.33",  # 0.1
        ################################################
        # obs
        ################################################
        f"observation.planner_type=pdm_closed",
        ################################################
        # obs occlusion
        ################################################
        f"observation.occlusion_cfg.occlusion=True",
        f"observation.occlusion_cfg.manager_type=wedge",
        f"+observation.occlusion_cfg.uncloak_reaction_time=0.5",
        f"+observation.occlusion_cfg.notice_threshold=0.33",
        ################################################
        # scenario modification
        "+modify_scenario_simulations=true",
        "+second_testing_round=true", 
        # unlike in the bulk running experiments ipynb file, we are testing all the scenarios.
        # #not just the ones that are OCCs, just in case different settings result in different scenarios having OCCs. 
        # If you want to restrict this, you can set the `scenarios_to_check` variable in the config like in the bulk running experiments file.
        f"+modification_file_path={modification_file_path}",
        # "+remove_other_agents=true",
        ################################################
        # misc
        ################################################
        "+simulation=closed_loop_multiagent",  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents, closed_loop_multiagent]
        "worker=ray_distributed",  # [sequential, ray_distributed]
        "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
        *DATASET_PARAMS,
        ################################################
    ],
)

output_folder_reg_2 = cfg_occ.output_dir
print('output_folder_reg_2 = "' + output_folder_reg_2 + '"')

### Run Second Regime

In [ ]:
# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
main_simulation(cfg_occ)

## Number of collisions of of each scenario type visualized. You may use the plot_diff function to visualize many more metrics

In [ ]:
from vis_utils import plot_diff

# %env NUPLAN_DATA_ROOT=../../../datasets/nuplan/dataset
# %env NUPLAN_MAPS_ROOT=../../data/nuplan/maps
# %env NUPLAN_EXP_ROOT=../../data/nuplan/exp
# %env NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/
# %pwd

file_path_1 = output_folder_reg_1 + "/metrics/" + "no_ego_at_fault_collisions.parquet"
file_path_2 = output_folder_reg_2 + "/metrics/" + "no_ego_at_fault_collisions.parquet"


metrics = [
    "number_of_ego_collisions_with_vehicles_stat_value",
]
metric_op = [
    "sum",
]
# metrics are 'good' if you would like to succeed at them. ex: amount of crashes is bad, amount of money is good
good_metric = [
    False,
]

plot_diff(
    file_path_2,
    file_path_1,
    "unocc",
    "occ",
    metrics,
    metric_op,
    good_metric,
    group_by="scenario_type",
    k_differences=25,
    as_bool=True,
)

## Prep NuBoard

In [ ]:
import re

output_folder = [output_folder_reg_1, output_folder_reg_2]

print(output_folder_reg_1)
print(output_folder_reg_2)

CONFIG_PATH = "../nuplan/planning/script/config/nuboard"
CONFIG_NAME = "default_nuboard"

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)
scenario_builder = None
for item in DATASET_PARAMS:
    parts = re.split("=", item, 1)
    if parts[0] == "scenario_builder":
        scenario_builder = parts[1]
if scenario_builder is None:
    raise ValueError("scenario_builder not specified in the dataset parameters")
print(scenario_builder)
# Compose the configuration
cfg = hydra.compose(
    config_name=CONFIG_NAME,
    overrides=[
        f"scenario_builder={scenario_builder}",  # set the database (same as simulation) used to fetch data for visualization
        f"simulation_path={output_folder}",  # [output_folder, output_folder_alt] nuboard file path(s), if left empty the user can open the file inside nuBoard
    ],
)

### Launch NuBoard

In [ ]:
from nuplan.planning.script.run_nuboard import main as main_nuboard

# Run nuBoard
main_nuboard(cfg)